In [6]:
#Data Collection



#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv


import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

In [7]:
#Preprocessing

allposts = pd.read_csv("allPostData.csv", header=0)

#allposts = allposts.dropna()

# Dropping multiple blank columns
# allposts = allposts.drop(["archivedStatus", 
#                           "created_at", 
#                           "created_at__timezone",
#                           "id", 
#                           "owner__about", 
#                           "owner__account_type",
#                           "owner__badge_count",
#                           "owner__buyer_rating",
#                           "owner__created_at",
#                           "owner__email",
#                           "owner__email_verified",
#                           "owner__expo_token",
#                           "owner__false_report_count",
#                           "owner__first_name",
#                           "owner__full_name",
#                           "owner__id",
#                           "owner__is_student",
#                           "owner__last_modified",
#                           "owner__last_name",
#                           "owner__passwordHash",
#                           "owner__profile_pic",
#                           "owner__provider_rating",
#                           "owner__report_count",
#                           "owner__school__address__components__city",
#                           "owner__school__address__components__country",
#                           "owner__school__address__components__county",
#                           "owner__school__address__components__number",
#                           "owner__school__address__components__state",
#                           "owner__school__address__components__street",
#                           "owner__school__address__components__zip_code",
#                           "owner__school__address__formatted",
#                           "owner__school__created_at",
#                           "owner__school__crest",
#                           "owner__school__domains__001",
#                           "owner__school__domains__002",
#                           "owner__school__id",
#                           "owner__school__last_modified",
#                           "owner__school__location__coordinates__001",
#                           "owner__school__location__coordinates__002",
#                           "owner__school__location__type",
#                           "owner__school__name",
#                           "owner__school__website",
#                           "owner__school_id",
#                           "owner__strikes_count",
#                           "owner__stripe_customer__account_balance",
#                           "owner__stripe_customer__created",
#                           "owner__stripe_customer__currency",
#                           "owner__stripe_customer__default_source",
#                           "owner__stripe_customer__delinquent",
#                           "owner__stripe_customer__description",
#                           "owner__stripe_customer__discount",
#                           "owner__stripe_customer__email",
#                           "owner__stripe_customer__id",
#                           "owner__stripe_customer__invoice_prefix",
#                           "owner__stripe_customer__livemode",
#                           "owner__stripe_customer__object",
#                           "owner__stripe_customer__shipping",
#                           "owner__stripe_customer__sources__has_more",
#                           "owner__stripe_customer__sources__object",
#                           "owner__stripe_customer__sources__total_count",
#                           "owner__stripe_customer__sources__url",
#                           "owner__stripe_customer__subscriptions__has_more",
#                           "owner__stripe_customer__subscriptions__object",
#                           "owner__stripe_customer__subscriptions__total_count",
#                           "owner__stripe_customer__subscriptions__url",
#                           "owner__stripe_customer__tax_info",
#                           "owner__stripe_customer__tax_info_verification",
#                           "owner__subscriptions",
#                           "photos__001",
#                           "photos__002",
#                           "photos__003",
#                           "photos__004"
#                          ], axis=1) 

#allposts.head()

In [8]:
# only select relevant columns

allposts = allposts[['description', 'title']]

In [9]:
allposts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 2 columns):
description    811 non-null object
title          811 non-null object
dtypes: object(2)
memory usage: 12.8+ KB


In [10]:
allposts.shape

(811, 2)

In [11]:
# drop null columns

allposts = allposts.dropna()

In [12]:
allposts = allposts['title']

In [13]:
allposts

0                      Teach Me How To Cook!
1                      Long Boarding Lessons
2                           Personal Trainer
3                       Final Cut Pro Lesson
4                            Resume Building
5                      Motivational Pep Talk
6                     Resume + Cover Letter 
7                          Paracord Bracelet
8                          Video Production 
9                           Consulting Cases
10                           SAS / SQL Tutor
11                         Video Production 
12                            Guitar Lessons
13                   Photography / Portraits
14                  Help Your Tinder Profile
15                      Custom Love Song 😍😍😍
16                                 Gym Buddy
17                      Personal Cheerleader
18                                       Fan
19                                 Lil Ducky
20                      Snowboarding Lessons
21                 Bed Bath & Beyond Drawers
22        

In [14]:
#Data Cleaning

#(1) Define a function that takes in a list of strings and cleans it up

#remove words in stoplist from texts

stoplist = set(stopwords.words('english'))
texts = [[word for word in document.lower().split() if word not in stoplist]
             for document in allposts]

#why does the order of commands matter so much?? check RS_4

In [15]:
#sanity check

print(stoplist)
print(texts)

{'mightn', 'yourselves', 'have', 'an', 'itself', "hadn't", 'during', 'same', 'did', 'hadn', 'hasn', 'its', 'was', 'down', 'such', 'between', "wouldn't", 'being', 'should', 'ours', 'shouldn', 'before', 'are', "should've", 'they', 'myself', 'no', 'below', 'doesn', 'which', 'this', 'haven', 'too', 'hers', 'nor', 'that', 'ourselves', 'but', 'both', "you'll", 'had', 'from', 'my', 'themselves', 'because', 'up', 'don', 'himself', "mightn't", 'a', 'our', 'herself', 'most', 'just', 'me', 'once', 'own', 'these', 'of', 'after', 'mustn', "aren't", 'them', "it's", 'through', 'aren', 'only', 'not', "hasn't", 'will', 'very', 're', 'were', 'by', 'to', "you're", 'some', 'doing', 'those', 'ma', 'wasn', "isn't", 'about', 'any', 'then', 'until', 'other', 'am', 'out', 'there', 'does', 'd', 's', "needn't", 'can', 'wouldn', 'more', 'been', 'you', 'here', 'm', "mustn't", 'while', 've', 'how', "weren't", 'over', 'why', 'for', 'y', "didn't", 'do', 'and', 'is', 'who', "shan't", "wasn't", 'weren', 'theirs', 'your

In [16]:
#remove punctuation from texts

txtre_clean = re.compile('<.*?/+!>')
new_clean = [[re.sub(txtre_clean, ' ', word) for word in document] for document in texts]

In [17]:
print(new_clean)
#Lol not working fml wtf

[['teach', 'cook!'], ['long', 'boarding', 'lessons'], ['personal', 'trainer'], ['final', 'cut', 'pro', 'lesson'], ['resume', 'building'], ['motivational', 'pep', 'talk'], ['resume', '+', 'cover', 'letter'], ['paracord', 'bracelet'], ['video', 'production'], ['consulting', 'cases'], ['sas', '/', 'sql', 'tutor'], ['video', 'production'], ['guitar', 'lessons'], ['photography', '/', 'portraits'], ['help', 'tinder', 'profile'], ['custom', 'love', 'song', '😍😍😍'], ['gym', 'buddy'], ['personal', 'cheerleader'], ['fan'], ['lil', 'ducky'], ['snowboarding', 'lessons'], ['bed', 'bath', '&', 'beyond', 'drawers'], ['math', 'tutor'], ['3', 'head', 'lamp'], ['web', 'development'], ['rice', 'cooking', 'pot'], ['get', 'startup', 'job'], ['tennis', 'racket'], ['alarm', 'clock', '/', 'clock'], ['orgo', 'kit'], ['cs', 'tutor'], ['ethernet', 'cable'], ['social', 'media', 'management'], ['mouse'], ['photography'], ['bed', 'raises'], ['tv', 'screen'], ['homecooked', 'dinner'], ['ig', 'social', 'media', 'marke

In [18]:
# printing a cleaned document (which is a list of lists) without stopwords
cleaned_up_document = texts
pprint(cleaned_up_document)


[['teach', 'cook!'],
 ['long', 'boarding', 'lessons'],
 ['personal', 'trainer'],
 ['final', 'cut', 'pro', 'lesson'],
 ['resume', 'building'],
 ['motivational', 'pep', 'talk'],
 ['resume', '+', 'cover', 'letter'],
 ['paracord', 'bracelet'],
 ['video', 'production'],
 ['consulting', 'cases'],
 ['sas', '/', 'sql', 'tutor'],
 ['video', 'production'],
 ['guitar', 'lessons'],
 ['photography', '/', 'portraits'],
 ['help', 'tinder', 'profile'],
 ['custom', 'love', 'song', '😍😍😍'],
 ['gym', 'buddy'],
 ['personal', 'cheerleader'],
 ['fan'],
 ['lil', 'ducky'],
 ['snowboarding', 'lessons'],
 ['bed', 'bath', '&', 'beyond', 'drawers'],
 ['math', 'tutor'],
 ['3', 'head', 'lamp'],
 ['web', 'development'],
 ['rice', 'cooking', 'pot'],
 ['get', 'startup', 'job'],
 ['tennis', 'racket'],
 ['alarm', 'clock', '/', 'clock'],
 ['orgo', 'kit'],
 ['cs', 'tutor'],
 ['ethernet', 'cable'],
 ['social', 'media', 'management'],
 ['mouse'],
 ['photography'],
 ['bed', 'raises'],
 ['tv', 'screen'],
 ['homecooked', 'dinne

In [19]:
#(2) Vectorize the document 

#define training data and train model (use pre-trained)

#Load Google's pre-trained Word2Vec 
#model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

#word_vectors = api.load("glove-wiki-gigaword-100")

#path = get_tmpfile("word2vec.model")
#model = Word2Vec(common_texts, min_count=1)
#model = Word2Vec(cleaned_up_document, min_count=1)
#model.save("word2vec.model")
#model.wv.save_word2vec_format('model.txt', binary=False)
#print(model)


#sentences - (iterable of list of str) – list of lists of tokens
#size - (int, optional) – Dimensionality of the word vectors
#window - (int, optional) – Maximum distance between the current
#and predicted word within a sentence
#min_count (int, optional) – Ignores all words with total 
#frequency lower than this.
#workers (int, optional) – Use these many worker threads to train the model (faster training with multicore machines



#loading wikipedia data 

corpus = api.load('text8')
model = Word2Vec(corpus)

In [20]:
# summarize the loaded model

info = api.info()
print(model)

Word2Vec(vocab=71290, size=100, alpha=0.025)


In [21]:
# summarize vocabulary (using wikipedia data http://mattmahoney.net/dc/textdata.html)

words = list(model.wv.vocab)
print(len(words))

#model.wv - object essentially contains the mapping between words and embeddings
#model = vectorized wikipedia pre-trained corpus 
#word_vectors = vectorized pangea data

71290


In [22]:
#sanity check 
vector = model.wv['computer']

In [23]:
print(model.wv.syn0.shape)

(71290, 100)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [24]:
#apply vectorization to test data
word_vectors = Word2Vec(cleaned_up_document)

In [25]:
#sanity check
print(word_vectors.wv['food'])

[-2.2717598e-03  1.5452105e-03  4.2319088e-03 -4.8308438e-03
 -4.2206538e-03  7.7883742e-04  4.4480287e-03  2.9691940e-03
 -3.4393190e-05 -3.7431458e-03  3.0682501e-03  2.6734942e-03
  4.4119996e-03 -2.8453327e-03 -4.1652960e-03  3.5442957e-03
 -3.8241546e-03 -4.7632819e-03 -2.7636709e-03  9.7699545e-04
 -4.3335264e-03  4.2703180e-03  1.0572422e-03 -4.8786025e-03
  2.8322262e-03 -9.6953177e-04  3.7837791e-04  7.8010675e-04
 -4.0229750e-03 -4.8245676e-03  2.9292444e-03 -3.4244866e-03
  1.6530784e-03  4.8096511e-03  3.4508924e-03 -1.1826849e-03
  4.2904746e-03 -3.3008340e-03 -4.1298047e-03 -1.5578254e-03
 -3.9288509e-03  7.8022212e-04 -2.9800185e-03 -1.8998327e-04
 -4.1957721e-03  5.3808716e-04 -3.3501808e-03  3.8863053e-03
  2.7022206e-03 -1.7474638e-03  9.2547853e-04  3.2131991e-03
 -1.6546011e-03 -1.9928180e-04 -3.1536710e-03  2.3597523e-03
  3.7400431e-03  2.1920651e-03  2.4649219e-03  1.5031623e-03
 -4.4440739e-03 -1.7416271e-03 -8.6208252e-04 -1.7472838e-04
 -1.1674876e-03 -1.60410

In [26]:
print(word_vectors)

#own_words = list(word_vectors.wv.vocab)
#own_words = word_vectors.wv.vocab
#print(own_words)
#seems like we're missing words here

word_vectors

Word2Vec(vocab=65, size=100, alpha=0.025)


In [40]:
#vectorized_document = np.average(array_words, axis = 0)

#goal: for each list of vectors in word_vectors, get the mean. 
#output: we'll have a vectorized document comprised of a list of vector means


# output is numpy 2D array of vectors
matrix_wordvec = word_vectors.wv.syn0

matrix_words = model.wv.syn0
#this is for getting the 2d numpy vector (?)


#vectorized_document = np.mean(matrix_wordvec, axis=1)
#taking the mean of each line --> how do I make sure this is right? 
#basically want the ave of each sentence list


list_of_sentence_values = []
for sentence in cleaned_up_document:
    sentiment = np.zeros(100) 
    for word in sentence:
        if word in word_vectors:
            sentiment += word_vectors.wv[word]
        else:
            sentiment += 0
    ave_sentiment = sentiment/len(sentence)
    list_of_sentence_values.append(ave_sentiment)
print(list_of_sentence_values)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).


[array([ 1.67591264e-03, -2.48040748e-03, -1.78385829e-03, -1.21664209e-03,
       -2.31083506e-03,  7.60754163e-04, -1.63905066e-03, -4.58789640e-04,
       -2.14668969e-03, -2.73445563e-04,  8.23684750e-05,  9.68269771e-04,
        2.45434023e-03, -1.81221892e-03, -5.51473677e-05,  3.98083757e-05,
        2.62132438e-04, -8.53605685e-04,  4.12550376e-04, -6.97271666e-04,
        1.84885910e-04,  7.28277781e-04,  1.74608931e-03, -1.34280964e-03,
       -1.70741137e-03,  1.66162115e-03, -6.13816315e-04, -1.81681651e-03,
       -6.77135249e-04, -5.55286300e-04,  7.22859695e-04,  2.18439509e-05,
       -1.24270329e-03,  1.67024395e-04,  5.99700725e-04,  6.23710337e-04,
        1.85013923e-03, -2.24836590e-03, -3.68829991e-04, -3.51132010e-04,
       -2.15908838e-03,  3.01276566e-04,  2.31244811e-03, -4.44350881e-04,
       -1.02223060e-03, -2.22358899e-03,  9.74505849e-04, -1.44364923e-04,
       -1.77133828e-03, -1.77524902e-03,  2.04144255e-03, -2.38482654e-03,
        1.46750687e-03, 

In [35]:
print(list_of_sentence_values.shape)

AttributeError: 'list' object has no attribute 'shape'

In [36]:
print(matrix_wordvec)
#is this right lols --> this is pangea data

[[-0.00018763 -0.00271352 -0.00344877 ...  0.00369099 -0.0038314
   0.000339  ]
 [-0.00094432  0.00307008 -0.00042013 ...  0.00042418  0.00365736
  -0.00010442]
 [-0.00290555 -0.00074495 -0.00473604 ... -0.00482772 -0.00135428
   0.00487104]
 ...
 [ 0.00434751 -0.00030269 -0.00130343 ... -0.00167812  0.0042088
  -0.00412701]
 [-0.00260405  0.00451686 -0.00354146 ... -0.00226339 -0.00188648
   0.00253323]
 [-0.00227176  0.00154521  0.00423191 ... -0.00118969 -0.00070846
  -0.00150232]]


In [41]:
print(matrix_words)
#this is the matrix of wiki data
print(matrix_words.shape)

print(matrix_wordvec.shape)

[[-1.5041667  -0.537313    0.63828987 ...  1.1577718  -1.1624347
   1.2336546 ]
 [-0.89003825  0.48970065  0.6459784  ...  0.1705134  -1.7929975
   0.48297304]
 [-0.54489154 -0.9847597  -0.03941699 ...  1.0566267  -0.46979904
   1.202166  ]
 ...
 [-0.0332514  -0.04654291  0.05920037 ...  0.03915379  0.07606809
   0.06456012]
 [-0.01999551 -0.10319483  0.06898763 ...  0.01542308  0.10715648
   0.02754806]
 [-0.03173968 -0.08740678  0.08707175 ...  0.06704281  0.13481551
   0.0091961 ]]
(71290, 100)
(65, 100)


In [38]:
print(vectorized_document)
print(vectorized_document.shape)
#this is the vectorized list of lists of sentences

NameError: name 'vectorized_document' is not defined

In [28]:
#(3) Find the most similar cosines using matrix dot product

#input_doc_vec: np.array with the vectorized sentence/word

input_doc_vec = matrix_words

#Docs_from_db_vec: list of np.arrays 
#(or an array of np arrays) of the vectorized sentences/words in the database

docs_from_db_vec = vectorized_document

In [41]:
# ranking similarities
docs_with_ranks = []

for db_doc_vec in docs_from_db_vec:
#for vectorized_sentence in input_doc_vec:
    similarity = np.dot(input_doc_vec, docs_from_db_vec)
    docs_with_ranks.append((similarity, docs_from_db_vec))
#pprint(docs_with_ranks[0])
    
#docs with ranks += --> we're adding 2 values, the similarity query and 
#the corresponding vectorized document ????
#basically want to output the similarity rankings and then the data 

In [42]:
pprint(docs_with_ranks)
#shouldn't the dot product be returning a single number? 
#this line seems wrong

[(array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32),
  array([ 1.1618879e-04, -7.4296311e-04,  4.1184720e-04,  5.0001318e-04,
       -8.5845400e-05,  3.6111849e-04, -3.3712893e-04, -5.2932995e-05,
       -2.1530471e-04, -1.3697769e-04, -2.4863734e-04,  3.5137226e-04,
        4.3463806e-04, -4.9058185e-04,  6.7570194e-04, -8.5597468e-04,
        1.2023843e-04, -5.3617032e-04, -1.3198577e-04, -8.0554921e-05,
        5.3257681e-04,  6.2418869e-05, -2.5719043e-04, -3.1853986e-05,
       -1.7020175e-04,  3.4561360e-04,  2.4831994e-04, -2.0890462e-04,
       -1.7973092e-04, -4.5414787e-04, -2.6509917e-04,  6.1060913e-04,
       -1.9089370e-04, -2.2547751e-05,  7.3083145e-05,  4.1685122e-05,
        8.8244215e-06,  5.0463248e-04, -2.3594385e-04, -5.0495702e-05,
       -4.6109208e-06,  2.7945390e-04,  1.0700180e-04,  2.4564902e-04,
        2.5501003e-04, -9.0510811e-04,  7.3309326e-05,  1.8066610e-04,
        3.78

 (array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32),
  array([ 1.1618879e-04, -7.4296311e-04,  4.1184720e-04,  5.0001318e-04,
       -8.5845400e-05,  3.6111849e-04, -3.3712893e-04, -5.2932995e-05,
       -2.1530471e-04, -1.3697769e-04, -2.4863734e-04,  3.5137226e-04,
        4.3463806e-04, -4.9058185e-04,  6.7570194e-04, -8.5597468e-04,
        1.2023843e-04, -5.3617032e-04, -1.3198577e-04, -8.0554921e-05,
        5.3257681e-04,  6.2418869e-05, -2.5719043e-04, -3.1853986e-05,
       -1.7020175e-04,  3.4561360e-04,  2.4831994e-04, -2.0890462e-04,
       -1.7973092e-04, -4.5414787e-04, -2.6509917e-04,  6.1060913e-04,
       -1.9089370e-04, -2.2547751e-05,  7.3083145e-05,  4.1685122e-05,
        8.8244215e-06,  5.0463248e-04, -2.3594385e-04, -5.0495702e-05,
       -4.6109208e-06,  2.7945390e-04,  1.0700180e-04,  2.4564902e-04,
        2.5501003e-04, -9.0510811e-04,  7.3309326e-05,  1.8066610e-04,
        3.78

In [43]:
pprint(similarity)

#sanity check --> ok, i guess it returns one number per vector pair 
#does it matter that dtype = float? 

array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32)


In [44]:
#docs_with_ranks.tolist()
#sorted(docs_with_ranks)
#docs_with_ranks.sort

#sorted((docs_with_ranks).all())

#sorted(docs_with_ranks, key=similarity)

#sorted(docs_with_ranks, key=attrgetter('similarity'))

#similarity_sort = [item for item in docs_with_ranks.sort()]  

# vectorized_document.similar_by_vector([1,2])

pprint(docs_with_ranks[0])


(array([-7.2760601e-04, -7.8558549e-03, -2.9154588e-03, ...,
        3.5451976e-05,  2.2109351e-04,  9.0466063e-05], dtype=float32),
 array([ 1.1618879e-04, -7.4296311e-04,  4.1184720e-04,  5.0001318e-04,
       -8.5845400e-05,  3.6111849e-04, -3.3712893e-04, -5.2932995e-05,
       -2.1530471e-04, -1.3697769e-04, -2.4863734e-04,  3.5137226e-04,
        4.3463806e-04, -4.9058185e-04,  6.7570194e-04, -8.5597468e-04,
        1.2023843e-04, -5.3617032e-04, -1.3198577e-04, -8.0554921e-05,
        5.3257681e-04,  6.2418869e-05, -2.5719043e-04, -3.1853986e-05,
       -1.7020175e-04,  3.4561360e-04,  2.4831994e-04, -2.0890462e-04,
       -1.7973092e-04, -4.5414787e-04, -2.6509917e-04,  6.1060913e-04,
       -1.9089370e-04, -2.2547751e-05,  7.3083145e-05,  4.1685122e-05,
        8.8244215e-06,  5.0463248e-04, -2.3594385e-04, -5.0495702e-05,
       -4.6109208e-06,  2.7945390e-04,  1.0700180e-04,  2.4564902e-04,
        2.5501003e-04, -9.0510811e-04,  7.3309326e-05,  1.8066610e-04,
        3.7857

In [45]:
np.sort(vectorized_document)

array([-9.0510811e-04, -8.5597468e-04, -7.4296311e-04, -6.4009405e-04,
       -6.1266019e-04, -5.7433994e-04, -5.3617032e-04, -5.1228155e-04,
       -5.0345092e-04, -4.9058185e-04, -4.8094519e-04, -4.5414787e-04,
       -4.5091278e-04, -4.1009716e-04, -4.0001990e-04, -3.5208036e-04,
       -3.3712893e-04, -3.2047502e-04, -3.1924038e-04, -3.1670128e-04,
       -2.8700178e-04, -2.8295862e-04, -2.7502768e-04, -2.7380852e-04,
       -2.7241724e-04, -2.6509917e-04, -2.5841800e-04, -2.5719043e-04,
       -2.4863734e-04, -2.4601928e-04, -2.3594385e-04, -2.3109549e-04,
       -2.1530471e-04, -2.0890462e-04, -1.9119307e-04, -1.9089370e-04,
       -1.7973092e-04, -1.7462514e-04, -1.7020175e-04, -1.6463583e-04,
       -1.6118876e-04, -1.5413226e-04, -1.4530991e-04, -1.4279278e-04,
       -1.3697769e-04, -1.3198577e-04, -1.0501011e-04, -8.5845400e-05,
       -8.0554921e-05, -5.9638951e-05, -5.2932995e-05, -5.0495702e-05,
       -3.1853986e-05, -2.2547751e-05, -1.4225365e-05, -4.6109208e-06,
      

In [46]:
np.any(sorted(docs_with_ranks))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [35]:
np.sort(docs_with_ranks, axis=0)

ValueError: setting an array element with a sequence.

def find_most_similar(input_doc_vec, docs_from_db_vec):
    input_doc_vec = vectorize_document(clean)
    docs_from_db_vec = vectorize_document(clean)
    #would these 2 be the same now? since we only have one dataset that hasn't been split into training/testing?
    docs_with_ranks = []
    for db_doc_vec in docs_from_db_vec: 
        similarity = np.dot(input_doc_vec, db_doc_vec)
        docs_with_ranks += [(similarity, docs_db_vec)] 
    return sort_by_similarity(docs_with_ranks)


def process_new_document(document): 
    ```document: a raw document that the user provides```
        cleaned_document = clean_raw_document(document)
        vectorized_document = vectorize_document(cleaned_document)
        docs_from_db_vec = load_from_db(filepath_to_csv_or_db)
        save(vectorized_document, filepath_to_csv_or_db) <- might be a bit more complicated, since you’ll want to save it into a table along with the other document vectors that your users have input earlier, along with an index of some kind that lets you look the vector up
        sorted_documents = find_most_similar(vectorized_document, docs_from_db_vec)

    return sorted_documents
